# Tutorial

An example of using Theatre_Ag to model the establishment of stateful TCP connections over an IP network.

In [ ]:
!pip install git+https://github.com/twsswt/theatre_ag

First, we define the simulation clock with a maximum tick:

In [ ]:
from theatre_ag import SynchronizingClock
clock = SynchronizingClock(3)

Define the cast:

In [ ]:
from theatre_ag import TaskQueueActor, Cast

tcp_client = TaskQueueActor('tcp_client', clock)
tcp_server = TaskQueueActor('tcp_server', clock)
cast = Cast({tcp_client, tcp_server})

Define the scene the cast will operate on (the IP network)

In [ ]:
from theatre_tcp_ip import Network
network = Network()

Finally we use the [<code>TCPDirections</code>](./theatre_tcp_ip/tcp_workflow.py) and an episode to link the scene and cast together and perform the simulation:

In [ ]:
from theatre_ag import Episode
from theatre_tcp_ip import TCPDirections

directions = TCPDirections(network, 'tcp_server')

episode = Episode(clock, cast, directions)

episode.perform()

It is possible to query the state of the simulation, include the task history of the actors:

In [ ]:
from theatre_ag import format_task_trees

print format_task_trees(tcp_client.task_history)
print format_task_trees(tcp_server.task_history)